## Data Loading and Cleaning

In [1]:
import pandas as pd
from pymongo import MongoClient

# Load the dataset
data = pd.read_csv('Resources/Impact_of_Remote_Work_on_Mental_Health.csv')
print("Dataset loaded successfully!")

# Display the first few rows to get a sense of the structure
data.head()

# Data Cleaning Function
def clean_data(df):
    # Handle missing values
    df['Mental_Health_Condition'] = df['Mental_Health_Condition'].fillna('None')
    df['Physical_Activity'] = df['Physical_Activity'].fillna('None')
    
    # Standardize categorical values
    df['Gender'] = df['Gender'].str.capitalize()
    df['Job_Role'] = df['Job_Role'].str.title().replace({'Hr': 'HR'})
    df['Industry'] = df['Industry'].str.title().replace({'It': 'IT'})
    df['Work_Location'] = df['Work_Location'].str.capitalize()
    df['Stress_Level'] = df['Stress_Level'].str.capitalize().replace({'Low ': 'Low'})
    df['Productivity_Change'] = df['Productivity_Change'].str.title()
    df['Satisfaction_with_Remote_Work'] = df['Satisfaction_with_Remote_Work'].str.title()
    df['Access_to_Mental_Health_Resources'] = df['Access_to_Mental_Health_Resources'].str.capitalize()
    df['Sleep_Quality'] = df['Sleep_Quality'].str.title()
    df['Region'] = df['Region'].str.title()
    
    # Handling outliers in 'Hours_Worked_Per_Week'
    df['Hours_Worked_Per_Week'] = df['Hours_Worked_Per_Week'].clip(lower=20, upper=60)
    
    return df

# Apply the cleaning function
data_cleaned = clean_data(data)

# Check the first few rows of cleaned data
data_cleaned.head()




Dataset loaded successfully!


,Employee_ID,Age,Gender,Job_Role,Industry,Years_of_Experience,Work_Location,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Mental_Health_Condition,Access_to_Mental_Health_Resources,Productivity_Change,Social_Isolation_Rating,Satisfaction_with_Remote_Work,Company_Support_for_Remote_Work,Physical_Activity,Sleep_Quality,Region
0,EMP0001,32,Non-binary,HR,Healthcare,13,Hybrid,47,7,2,Medium,Depression,No,Decrease,1,Unsatisfied,1,Weekly,Good,Europe
1,EMP0002,40,Female,Data Scientist,IT,3,Remote,52,4,1,Medium,Anxiety,No,Increase,3,Satisfied,2,Weekly,Good,Asia
2,EMP0003,59,Non-binary,Software Engineer,Education,22,Hybrid,46,11,5,Medium,Anxiety,No,No Change,4,Unsatisfied,5,None,Poor,North America
3,EMP0004,27,Male,Software Engineer,Finance,20,Onsite,32,8,4,High,Depression,Yes,Increase,3,Unsatisfied,3,None,Poor,Europe
4,EMP0005,49,Male,Sales,Consulting,32,Onsite,35,12,2,High,None,Yes,Decrease,3,Unsatisfied,3,Weekly,Average,North America


## Data Insertion into MongoDB

In [2]:
# MongoDB Connection and Data Insertion
client = MongoClient('mongodb://localhost:27017/')
db = client['remote_work_db']
collection = db['mental_health']

# Clear the collection and insert cleaned data into MongoDB
collection.delete_many({})
collection.insert_many(data_cleaned.to_dict('records'))
print("Data loaded into MongoDB successfully!")


Data loaded into MongoDB successfully!


## Exploratory Data Analysis (EDA) and Prototyping Visualizations with Plotly

#### Mental Health Condition by Job Role

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

# Group data to get counts for each Mental Health Condition
mental_health_counts = data_cleaned.groupby('Mental_Health_Condition').size().reset_index(name='count')

# Create a data source for Bokeh
source = ColumnDataSource(mental_health_counts)

# Create a Bokeh figure for the bar chart
p = figure(x_range=mental_health_counts['Mental_Health_Condition'], title="Mental Health Conditions by Job Role",
           toolbar_location=None, tools="")

# Add bars to the chart
p.vbar(x='Mental_Health_Condition', top='count', width=0.9, source=source)

# Customize axes and layout
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.axis_label = "Mental Health Condition"
p.yaxis.axis_label = "Number of Employees"
p.xaxis.major_label_orientation = "vertical"

# Show the plot in the notebook
show(p)


Loading BokehJS ...

#### Work-Life Balance vs. Hours Worked

In [9]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap

# Prepare the data
source = ColumnDataSource(data_cleaned)

# Create a color map for Stress Levels
stress_levels = data_cleaned['Stress_Level'].unique().tolist()
color_map = factor_cmap('Stress_Level', palette=['#ff9999', '#66b3ff', '#99ff99'], factors=stress_levels)

# Create a scatter plot with Bokeh
p2 = figure(title="Work-Life Balance vs. Hours Worked", x_axis_label='Hours Worked Per Week',
            y_axis_label='Work-Life Balance Rating')

# Add circles to the scatter plot
p2.circle(x='Hours_Worked_Per_Week', y='Work_Life_Balance_Rating', source=source, size=8, color=color_map, legend_field="Stress_Level")

# Customize the layout
p2.legend.title = "Stress Level"
p2.legend.location = "top_left"

# Show the plot in the notebook
show(p2)


#### Productivity Change by Mental Health Condition

In [10]:
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10

# Prepare data for productivity change by mental health condition
productivity_counts = data_cleaned.groupby(['Mental_Health_Condition', 'Productivity_Change']).size().unstack().fillna(0)

# Create a new data source
mental_health_conditions = productivity_counts.index.tolist()
source = ColumnDataSource(data=dict(
    mental_health_conditions=mental_health_conditions,
    Decrease=productivity_counts['Decrease'].values,
    Increase=productivity_counts['Increase'].values,
    No_Change=productivity_counts['No Change'].values
))

# Create a Bokeh figure for the stacked bar chart
p3 = figure(x_range=mental_health_conditions, title="Productivity Change by Mental Health Condition",
            toolbar_location=None, tools="")

# Add stacked bars to the chart
p3.vbar_stack(['Decrease', 'Increase', 'No_Change'], x='mental_health_conditions', width=0.9, color=Category10[3],
              source=source, legend_label=['Decrease', 'Increase', 'No Change'])

# Customize axes and layout
p3.y_range.start = 0
p3.xgrid.grid_line_color = None
p3.xaxis.axis_label = "Mental Health Condition"
p3.yaxis.axis_label = "Number of Employees"
p3.legend.title = "Productivity Change"
p3.legend.location = "top_left"

# Show the plot in the notebook
show(p3)


## Summary Statistics

In [6]:
# Numerical Summary Statistics
print("Numerical Summary Statistics:")
print(data_cleaned.describe())

# Categorical Summary Statistics
print("Categorical Summary Statistics:")
print(data_cleaned.describe(include='object'))


Numerical Summary Statistics:
               Age  Years_of_Experience  Hours_Worked_Per_Week  \
count  5000.000000          5000.000000            5000.000000   
mean     40.995000            17.810200              39.614600   
std      11.296021            10.020412              11.860194   
min      22.000000             1.000000              20.000000   
25%      31.000000             9.000000              29.000000   
50%      41.000000            18.000000              40.000000   
75%      51.000000            26.000000              50.000000   
max      60.000000            35.000000              60.000000   

       Number_of_Virtual_Meetings  Work_Life_Balance_Rating  \
count                 5000.000000               5000.000000   
mean                     7.559000                  2.984200   
std                      4.636121                  1.410513   
min                      0.000000                  1.000000   
25%                      4.000000                  2.000000 